In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import json
import re

In [11]:
juegos = pd.read_csv('/content/drive/MyDrive/games.csv')
items = pd.read_csv('/content/drive/MyDrive/items.csv')
reviews = pd.read_csv('/content/drive/MyDrive/reviews.csv')

In [12]:
patron = re.compile(r'\d+.*\d*')

juegos.release_date = pd.to_datetime(juegos.release_date)
juegos['año'] = juegos.release_date.dt.year
juegos["free"] = juegos.price.apply(lambda x: "free to play".find(x.lower().strip()) >= 0)
juegos["price"] = juegos.price.apply(lambda x: float(x) if patron.fullmatch(x) else 0)
juegos["price"] = juegos.price.astype(float, errors='ignore')

In [4]:
print(items.head())

   item_id  playtime_forever            user_id
0       10                 6  76561197970982479
1       20                 0  76561197970982479
2       30                 7  76561197970982479
3       40                 0  76561197970982479
4       50                 0  76561197970982479


In [5]:
print(reviews.head())

   item_id  recommend                                             review  \
0     1250       True  Simple yet with great replayability. In my opi...   
1    22200       True               It's unique and worth a playthrough.   
2    43110       True  Great atmosphere. The gunplay can be a bit chu...   
3   251610       True  I know what you think when you see this title ...   
4   227300       True  For a simple (it's actually not all that simpl...   

             user_id  sentiment_analysis  
0  76561197970982479                   2  
1  76561197970982479                   1  
2  76561197970982479                   2  
3            js41637                   2  
4            js41637                   2  


In [13]:
"""
  Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora
"""
def developer(developer: str):
  #Obtener la información de los juegos
  df = juegos[juegos["developer"] == developer][["año", "app_name", "free"]]

  #Agrupar por año y contar la cantidad de items y
  conteo = df.groupby("año")["app_name"].count().reset_index().sort_values(by="año")
  conteo.año = conteo.año.astype(int)

  #Agrupar por año y contar el porcentaje de Free to Play
  free = df.groupby("año")["free"].sum().reset_index().sort_values(by="año")
  free.año = free.año.astype(int)

  #Unir los dos dataframes
  conteo = pd.merge(conteo, free, on="año")
  conteo.free = round(conteo.free/conteo.app_name*100,2)
  return conteo

developer("Valve")

,año,app_name,free
0,1998,1,0.00
1,1999,1,0.00
2,2000,2,0.00
3,2001,1,0.00
4,2003,1,0.00
5,2004,5,0.00
6,2005,1,0.00
7,2006,2,0.00
8,2007,3,33.33
9,2008,1,0.00


In [14]:
"""
  cantidad de dinero gastado por el usuario, porcentaje de recomendación en base a reviews.recommend y cantidad de items
"""
def userdata(user: str):
  #Obtener la información del usuario y los items
  df = pd.merge(juegos, items, on='item_id')
  df = df[df["user_id"] == user]
  df = pd.merge(df, reviews, on=['item_id', 'user_id'])
  df = df[["price", "app_name", "recommend"]]

  #Obtener el total de dinero gastado, número de items y número de items recomendados
  precio = df.price.sum()
  cantidad = df.app_name.count()
  recomendacion = df.recommend.sum() / cantidad * 100

  return {
      "Usuario": user,
      "Dinero gastado": f"{precio} USD",
      "Cantidad de items": cantidad,
      "Porcentaje de recomendación": f"{recomendacion}%"
  }

userdata("js41637")

{'Usuario': 'js41637',
 'Dinero gastado': '29.979999999999997 USD',
 'Cantidad de items': 2,
 'Porcentaje de recomendación': '100.0%'}

In [15]:
"""
  Devolver el  año con más horas jugadas para un género dado.
  Input: genero, string
  Output: año, dict
"""
def play_time_genre(genero: str):
  #Obtener la información de los juegos y los items (que contiene las horas jugadas por juego y por usuario)
  df = pd.merge(juegos, items, on='item_id')

  # - Filtrar por el género dado y seleccionar las columnas año y playtime_forever (horas totales jugadas)
  # - Agrupar por año y sumar las horas jugadas por año
  # - Ordenar por horas jugadas de mayor a menor y seleccionar el primer elemento (año con más horas jugadas)
  df = df[df.genres.str.contains(genero)][["año", "playtime_forever"]]\
  .groupby("año").sum()\
  .sort_values(by="playtime_forever", ascending=False)\
  .reset_index()
  df.año = df.año.astype(int)

  #Año con más horas jugadas
  a = int(df.iloc[0]["año"])

  #Retorno
  return {f"Año de lanzamiento con más horas jugadas para Género {genero}":a}

play_time_genre("Action")

{'Año de lanzamiento con más horas jugadas para Género Action': 2012}

In [20]:
"""
|  Devolver el usuario con más horas jugadas para un género dado.
|  Input: genero, string
|  Output: usuario, dict
"""
def user_for_genre(genero: str):
  #Obtener la información de los juegos y los items (que contiene las horas jugadas por juego y por usuario)
  df = pd.merge(juegos, items, on='item_id')

  # - Filtrar por el género dado y seleccionar las columnas user_id, año y playtime_forever (horas totales jugadas)
  # - Agrupar por año y sumar las horas jugadas por año
  df = df[df.genres.str.contains(genero)][["user_id", "año", "playtime_forever"]]\
  .groupby(["año", "user_id"]).sum().reset_index()
  df["año"] = df["año"].astype(int)

  # Obtener la suma del tiempo jugado por usuario
  sum_usr = df.groupby("user_id")["playtime_forever"].sum().sort_values(ascending=False)
  # Obtener el usuario con más horas jugadas
  max_usr = sum_usr.idxmax()

  #Obtener las horas jugadas por el usuario en cada año y ordenar por año
  df = df[df.user_id == max_usr].loc[:,["año", "playtime_forever"]]\
  .set_index("año").sort_index()
  horas = [{"Año":row.año, "Horas":row.playtime_forever} for row in df.reset_index().itertuples()]

  #Retornar el usuario y las horas jugadas por año como un diccionario
  return {
      "Usuario con más horas jugadas": f"{max_usr}",
      "Horas jugadas por año": horas
  }

user_for_genre("Action")

{'Usuario con más horas jugadas': 'Sp3ctre',
 'Horas jugadas por año': [{'Año': 1970, 'Horas': 1657},
  {'Año': 1993, 'Horas': 0},
  {'Año': 1995, 'Horas': 217},
  {'Año': 1996, 'Horas': 0},
  {'Año': 1998, 'Horas': 0},
  {'Año': 1999, 'Horas': 44},
  {'Año': 2000, 'Horas': 70644},
  {'Año': 2001, 'Horas': 13},
  {'Año': 2002, 'Horas': 238},
  {'Año': 2003, 'Horas': 7673},
  {'Año': 2004, 'Horas': 127411},
  {'Año': 2005, 'Horas': 21339},
  {'Año': 2006, 'Horas': 896},
  {'Año': 2007, 'Horas': 112784},
  {'Año': 2008, 'Horas': 224},
  {'Año': 2009, 'Horas': 108326},
  {'Año': 2010, 'Horas': 78083},
  {'Año': 2011, 'Horas': 154896},
  {'Año': 2012, 'Horas': 379522},
  {'Año': 2013, 'Horas': 202746},
  {'Año': 2014, 'Horas': 130691},
  {'Año': 2015, 'Horas': 313172},
  {'Año': 2016, 'Horas': 29576},
  {'Año': 2017, 'Horas': 43327}]}

In [21]:
"""
  Top 3 de juegos recomendados por usuarios para el año dado.
  Input: año, int
"""
def users_recommend(año: int):
  #Obtener la información de los juegos y las reviews (que contiene las recomendaciones y el análisis de sentimientos)
  df = pd.merge(juegos, reviews, on='item_id')

  #Obtener las columnas necesarias
  df = df[df["año"] == año][["app_name", "recommend", "sentiment_analysis"]]
  #Filtrar los juegos recomendados y con reseñas positivas o neutrales
  mask = (df["recommend"] == True) & (df["sentiment_analysis"] > 0)
  df = df[mask]

  #Contar las recomendaciones por juego y ordenar
  df = df.groupby("app_name").count().sort_values(by="sentiment_analysis", ascending=False).reset_index()

  #Guardar los tres juegos más recomendados y retornar
  return [
      {"Puesto 1": f"{df.iloc[0]['app_name']}"},
      {"Puesto 2": f"{df.iloc[1]['app_name']}"},
      {"Puesto 3": f"{df.iloc[2]['app_name']}"}
  ]

users_recommend(2018)

[{'Puesto 1': 'Subnautica'},
 {'Puesto 2': 'DCS World'},
 {'Puesto 3': 'VR Regatta - The Sailing Game'}]

In [22]:
"""
  Top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado.
  Input: año, int
"""
def best_developer_year(año: int):
  #Obtener la información de los juegos y las reviews (que contiene las recomendaciones y el análisis de sentimientos)
  df = pd.merge(juegos, reviews, on='item_id')

  #Obtener las columnas necesarias
  df = df[df["año"] == año][["developer", "recommend", "sentiment_analysis"]]
  #Filtrar los juegos recomendados y con reseñas positivas o neutrales
  mask = (df["recommend"] == True) & (df["sentiment_analysis"] > 0)
  df = df[mask]

  #Contar las recomendaciones por desarrollador y ordenar
  df = df.groupby("developer").count().sort_values(by="sentiment_analysis", ascending=False).reset_index()
  #Guardar los tres desarrolladores más recomendados y retornar
  return [
      {"Puesto 1": f"{df.iloc[0]['developer']}"},
      {"Puesto 2": f"{df.iloc[1]['developer']}"},
      {"Puesto 3": f"{df.iloc[2]['developer']}"}
  ]

best_developer_year(2018)

[{'Puesto 1': 'Unknown Worlds Entertainment '},
 {'Puesto 2': 'Eagle Dynamics SA '},
 {'Puesto 3': 'MarineVerse '}]

In [23]:
"""
  Resumen de la cantidad de reseñas positivas, neutrales y negativas por año.
  Input: año, int
"""
def developer_reviews_analysis(dev: str):
  #Obtener la información de los juegos y las reviews (que contiene las recomendaciones y el análisis de sentimientos)
  df = pd.merge(juegos, reviews, on='item_id')

  #Obtener las columnas necesarias
  df = df[df["developer"] == dev][["sentiment_analysis"]].value_counts()
  df.index = ["Positive", "Neutral", "Negative"]

  #Retornar el resumen como un diccionario
  puntajes = []
  for i in df.index:
    puntajes.append(f"{i} = {df[i]}")
  return {f"{dev}":puntajes}

developer_reviews_analysis("Valve")

{'Valve': ['Positive = 5054', 'Neutral = 3825', 'Negative = 703']}